# Adaptive Cards Toolkit: 05d - Profile and Detailed Information Templates

This notebook demonstrates how to create profile templates and detailed information displays for Adaptive Cards. It continues from where 05c_toolkit_rich_media.ipynb left off, with a focus on specialized content types. Please run 01_toolkit_common.ipynb first.

## Setup

First, we need to import the common utilities defined in the toolkit_common notebook as well as all necessary dependencies.

In [ ]:
# Run common utilities notebook (optional, can be skipped if already run in session)
%run 01_toolkit_common.ipynb

# Ensure we have all the necessary imports
import json
import sys
import os
from datetime import datetime

# Add the parent directory to the path so we can import the toolkit
if not os.path.abspath('..') in sys.path:
    sys.path.insert(0, os.path.abspath('..'))

# Import necessary modules directly
from src.adaptive_cards_toolkit.core.element_factory import ElementFactory
from src.adaptive_cards_toolkit.core.layout_helper import LayoutHelper
from src.adaptive_cards_toolkit.templates.templates import TemplateFactory
from src.adaptive_cards_toolkit.core.data_connector import DataConnector

# Define our own validation utility since the imported one might have issues
class SimpleValidator:
    def validate(self, card):
        """Simple validation for cards - just calculates size"""
        try:
            card_json = card.to_json()
            size_kb = len(card_json) / 1024.0
            return {
                "valid": True,
                "size": size_kb,
                "details": [],
                "warnings": []
            }
        except Exception as e:
            return {
                "valid": False,
                "size": 0,
                "details": [str(e)],
                "warnings": []
            }

# Create an instance of our simple validator
simple_validator = SimpleValidator()

# Define display_card function for consistent rendering
def display_card(card, title=None):
    """Display a card as JSON with basic validation.
    
    Args:
        card: The card object to display
        title: Optional title to display with the card
    """
    if title:
        print(f"\n## {title}")
    try:
        # Get JSON from the card
        if hasattr(card, 'to_json'):
            card_json = json.loads(card.to_json())
        elif hasattr(card, 'card_data'):
            card_json = card.card_data
        else:
            card_json = card
            
        # Print the JSON
        print(json.dumps(card_json, indent=2))
        
        # Validate card
        if hasattr(card, 'to_json'):
            validation = simple_validator.validate(card)
            print(f"\nCard size: {validation['size']:.2f}KB | Valid: {validation['valid']}")
    except Exception as e:
        print(f"Error displaying card: {e}")

## Learning Objectives

In this notebook, you'll learn how to:

1. Create content with detailed metadata displays
2. Build profile cards for people and teams
3. Create employee directory cards for staff lookup
4. Design comprehensive templates for detailed information

## Content with Metadata

Now let's create a content template that prominently displays author information and other metadata:

In [ ]:
def create_content_with_metadata_template(title, content, metadata=None, image_url=None, actions=None):
    """Create a content template with rich metadata display.
    
    Args:
        title (str): Content title
        content (str): Main content text
        metadata (dict, optional): Dictionary of metadata (author, date, categories, etc.)
        image_url (str, optional): URL to a header image
        actions (list, optional): List of action dictionaries
        
    Returns:
        Object: Card object with to_json method
    """
    try:
        # Create card data with JSON construction pattern from style guide
        card_data = {
            "type": "AdaptiveCard",
            "version": "1.5",
            "body": []
        }
        
        # Add header image if provided
        if image_url:
            card_data["body"].append({
                "type": "Image",
                "url": image_url,
                "altText": f"Image for {title}",
                "size": "stretch"
            })
        
        # Add title
        card_data["body"].append({
            "type": "TextBlock",
            "text": title,
            "size": "extraLarge",
            "weight": "bolder",
            "wrap": True
        })
        
        # Add metadata in a structured format if provided
        if metadata and isinstance(metadata, dict):
            # Create metadata container
            metadata_container = {
                "type": "Container",
                "style": "emphasis",
                "padding": "default",
                "items": []
            }
            
            # Author with avatar if available
            if 'author' in metadata or 'author_avatar' in metadata:
                author_container = {
                    "type": "ColumnSet",
                    "columns": []
                }
                
                # Avatar column
                if 'author_avatar' in metadata:
                    author_container["columns"].append({
                        "type": "Column",
                        "width": "auto",
                        "items": [
                            {
                                "type": "Image",
                                "url": metadata['author_avatar'],
                                "size": "small",
                                "style": "person",
                                "altText": f"Avatar for {metadata.get('author', 'author')}"
                            }
                        ]
                    })
                
                # Author info column
                author_info = {
                    "type": "Column",
                    "width": "stretch",
                    "items": []
                }
                
                if 'author' in metadata:
                    author_info["items"].append({
                        "type": "TextBlock",
                        "text": metadata['author'],
                        "weight": "bolder",
                        "wrap": True
                    })
                
                # Author title/role if available
                if 'author_title' in metadata:
                    author_info["items"].append({
                        "type": "TextBlock",
                        "text": metadata['author_title'],
                        "isSubtle": True,
                        "size": "small",
                        "wrap": True
                    })
                
                author_container["columns"].append(author_info)
                metadata_container["items"].append(author_container)
            
            # Publication details
            pub_details = []
            
            if 'date' in metadata:
                pub_details.append(f"Published: {metadata['date']}")
                
            if 'reading_time' in metadata:
                pub_details.append(f"{metadata['reading_time']} min read")
                
            if 'categories' in metadata and metadata['categories']:
                if isinstance(metadata['categories'], list):
                    categories = ', '.join(metadata['categories'])
                    pub_details.append(f"Categories: {categories}")
                else:
                    pub_details.append(f"Category: {metadata['categories']}")
            
            if pub_details:
                metadata_container["items"].append({
                    "type": "TextBlock",
                    "text": ' · '.join(pub_details),
                    "isSubtle": True,
                    "size": "small",
                    "wrap": True
                })
            
            card_data["body"].append(metadata_container)
        
        # Add content with paragraph spacing
        paragraphs = content.split('\n\n')
        for paragraph in paragraphs:
            card_data["body"].append({
                "type": "TextBlock",
                "text": paragraph,
                "wrap": True
            })
        
        # Add actions if provided
        if actions:
            card_data["actions"] = []
            for action in actions:
                if action.get('type') == 'open_url':
                    card_data["actions"].append({
                        "type": "Action.OpenUrl",
                        "title": action.get('title', 'Open'),
                        "url": action.get('url', '#')
                    })
                elif action.get('type') == 'submit':
                    card_data["actions"].append({
                        "type": "Action.Submit",
                        "title": action.get('title', 'Submit'),
                        "data": action.get('data', {})
                    })
        
        # Create a custom card object with to_json method
        class CardObject:
            def __init__(self, card_data):
                self.card_data = card_data
                
            def to_json(self):
                return json.dumps(self.card_data)
        
        return CardObject(card_data)
    
    except Exception as e:
        print(f"Error creating content with metadata template: {e}")
        return None

Let's create a content article with rich metadata:

In [ ]:
try:
    metadata_article = create_content_with_metadata_template(
        title="Best Practices for AI-Generated Content Cards",
        content="When using AI to generate adaptive cards, it's important to follow certain best practices to ensure the content is accessible, relevant, and engaging.\n\nFirst, ensure that all generated cards include proper alt text for images and follow accessibility guidelines. AI systems should be trained to provide these automatically.\n\nSecond, personalize content based on user context whenever possible. This creates a more engaging experience and makes the information more relevant to the individual user.\n\nFinally, implement a feedback loop to improve card generation over time. User interactions with cards can provide valuable data on what works and what doesn't.",
        metadata={
            "author": "Dr. Maya Rodriguez",
            "author_title": "AI Communications Lead",
            "author_avatar": "https://adaptivecards.io/content/adaptive-card-50.png",
            "date": "February 23, 2023",
            "reading_time": "4",
            "categories": ["AI", "Design", "Best Practices"]
        },
        image_url="https://adaptivecards.io/content/adaptive-card-50.png",
        actions=[
            {
                "type": "open_url",
                "title": "View Full Article",
                "url": "https://example.com/articles/ai-content-cards"
            },
            {
                "type": "submit",
                "title": "Save for Later",
                "data": {"action": "save_article", "id": "ai-content-cards"}
            }
        ]
    )
    
    display_card(metadata_article, "Article with Rich Metadata")
    
except Exception as e:
    print(f"Error creating metadata article: {e}")

## Team Profiles

Now let's create a template for displaying team information with multiple members:

In [ ]:
def create_team_profile_template(team_name, team_description, members, team_image_url=None, actions=None):
    """Create a team profile card showing multiple team members.
    
    Args:
        team_name (str): Name of the team
        team_description (str): Description of the team or its purpose
        members (list): List of team member dictionaries, each containing name, title, image_url
        team_image_url (str, optional): URL to a team photo or logo
        actions (list, optional): List of action dictionaries
        
    Returns:
        Object: Card object with to_json method
    """
    try:
        # Initialize card data directly using JSON structure
        card_data = {
            "type": "AdaptiveCard",
            "version": "1.5",
            "body": []
        }
        
        # Create team header
        header = {
            "type": "Container",
            "style": "accent",
            "items": []
        }
        
        # Create columns for team header layout
        column_set = {
            "type": "ColumnSet",
            "columns": []
        }
        
        # Team image column if provided
        if team_image_url:
            image_column = {
                "type": "Column",
                "width": "auto",
                "items": [
                    {
                        "type": "Image",
                        "url": team_image_url,
                        "size": "medium",
                        "altText": f"Team image for {team_name}"
                    }
                ]
            }
            column_set["columns"].append(image_column)
        
        # Team details column
        details_column = {
            "type": "Column",
            "width": "stretch",
            "items": [
                {
                    "type": "TextBlock",
                    "text": team_name,
                    "size": "large",
                    "weight": "bolder",
                    "wrap": True
                },
                {
                    "type": "TextBlock",
                    "text": team_description,
                    "wrap": True
                }
            ]
        }
        column_set["columns"].append(details_column)
        
        # Add column set to header
        header["items"].append(column_set)
        
        # Add header to card body
        card_data["body"].append(header)
        
        # Add team members section header
        card_data["body"].append({
            "type": "TextBlock",
            "text": "Team Members",
            "size": "medium",
            "weight": "bolder",
            "wrap": True
        })
        
        # Create a container for all members
        members_container = {
            "type": "Container",
            "items": []
        }
        
        # Add each team member
        for i, member in enumerate(members):
            # Create member row with columns
            member_columns = []
            
            # Member image column
            if 'image_url' in member:
                image_column = {
                    "type": "Column",
                    "width": "auto",
                    "items": [
                        {
                            "type": "Image",
                            "url": member['image_url'],
                            "size": "small",
                            "style": "person",
                            "altText": f"Profile image for {member.get('name', 'Team member')}"
                        }
                    ]
                }
                member_columns.append(image_column)
            
            # Member details column
            member_details = []
            
            if 'name' in member:
                member_details.append({
                    "type": "TextBlock",
                    "text": member['name'],
                    "weight": "bolder",
                    "wrap": True
                })
            
            if 'title' in member:
                member_details.append({
                    "type": "TextBlock",
                    "text": member['title'],
                    "isSubtle": True,
                    "wrap": True
                })
                
            if 'email' in member:
                member_details.append({
                    "type": "TextBlock",
                    "text": member['email'],
                    "size": "small",
                    "wrap": True
                })
                
            if 'skills' in member and member['skills']:
                if isinstance(member['skills'], list):
                    skills_text = ", ".join(member['skills'])
                else:
                    skills_text = member['skills']
                    
                member_details.append({
                    "type": "TextBlock",
                    "text": f"Skills: {skills_text}",
                    "size": "small",
                    "wrap": True
                })
            
            details_column = {
                "type": "Column",
                "width": "stretch",
                "items": member_details
            }
            member_columns.append(details_column)
            
            # Create member column set
            member_row = {
                "type": "ColumnSet",
                "columns": member_columns
            }
            
            # Create member container
            member_item = {
                "type": "Container",
                "items": [member_row]
            }
            
            # Add separator between members (except after the last one)
            if i < len(members) - 1:
                member_item["items"].append({"type": "Separator"})
            
            # Add member to members container
            members_container["items"].append(member_item)
        
        # Add members container to card
        card_data["body"].append(members_container)
        
        # Add actions if provided
        if actions:
            card_data["actions"] = []
            for action in actions:
                if action.get('type') == 'open_url':
                    card_data["actions"].append({
                        "type": "Action.OpenUrl",
                        "title": action.get('title', 'Open'),
                        "url": action.get('url', '#')
                    })
                elif action.get('type') == 'submit':
                    card_data["actions"].append({
                        "type": "Action.Submit",
                        "title": action.get('title', 'Submit'),
                        "data": action.get('data', {})
                    })
        
        # Create a card object with to_json method
        class CardObject:
            def __init__(self, card_data):
                self.card_data = card_data
                
            def to_json(self):
                return json.dumps(self.card_data)
        
        return CardObject(card_data)
    
    except Exception as e:
        print(f"Error creating team profile template: {e}")
        return None

Let's create an example team profile card:

In [ ]:
try:
    # Sample team data
    team_members = [
        {
            "name": "Alex Johnson",
            "title": "Engineering Lead",
            "image_url": "https://adaptivecards.io/content/adaptive-card-50.png",
            "email": "alex.johnson@example.com",
            "skills": ["Python", "Cloud Architecture", "CI/CD"]
        },
        {
            "name": "Morgan Lee",
            "title": "Senior Developer",
            "image_url": "https://adaptivecards.io/content/adaptive-card-50.png", 
            "email": "morgan.lee@example.com",
            "skills": ["JavaScript", "React", "Mobile Development"]
        },
        {
            "name": "Taylor Smith",
            "title": "UX Designer",
            "image_url": "https://adaptivecards.io/content/adaptive-card-50.png",
            "email": "taylor.smith@example.com",
            "skills": ["UI/UX Design", "Design Systems", "Prototyping"]
        }
    ]
    
    # Create team profile card
    team_card = create_team_profile_template(
        team_name="Product Development Team",
        team_description="Cross-functional team responsible for the development and design of our flagship product suite.",
        members=team_members,
        team_image_url="https://adaptivecards.io/content/adaptive-card-50.png",
        actions=[
            {
                "type": "open_url",
                "title": "View Team Projects",
                "url": "https://example.com/teams/product-dev"
            },
            {
                "type": "submit",
                "title": "Contact Team",
                "data": {"action": "contact_team", "team_id": "product-dev"}
            }
        ]
    )
    
    display_card(team_card, "Team Profile Card")
    
except Exception as e:
    print(f"Error creating team profile card: {e}")

## Employee Directory Card

Now let's create a template for an employee directory that can be used for staff lookup:

In [ ]:
def create_employee_directory_template(title, employees, department_filter=None, search_box=True):
    """Create an employee directory card for staff lookup.
    
    Args:
        title (str): Title for the directory
        employees (list): List of employee dictionaries 
        department_filter (bool, optional): Whether to include department filter
        search_box (bool, optional): Whether to include a search box
        
    Returns:
        Object: Card object with to_json method
    """
    try:
        # Initialize card data using direct JSON
        card_data = {
            "type": "AdaptiveCard",
            "version": "1.5",
            "body": []
        }
        
        # Add title
        card_data["body"].append({
            "type": "TextBlock",
            "text": title,
            "size": "large",
            "weight": "bolder",
            "wrap": True
        })
        
        # Add search box if requested
        if search_box:
            search_container = {
                "type": "Container",
                "items": [
                    {
                        "type": "TextBlock",
                        "text": "Search Directory:",
                        "weight": "bolder",
                        "wrap": True
                    },
                    {
                        "type": "Input.Text",
                        "id": "search_query",
                        "placeholder": "Enter name, title, or skill"
                    }
                ]
            }
            card_data["body"].append(search_container)
            
            # Add actions array if it doesn't exist
            if "actions" not in card_data:
                card_data["actions"] = []
                
            # Add search button
            card_data["actions"].append({
                "type": "Action.Submit",
                "title": "Search",
                "data": {"action": "search_directory"}
            })
        
        # Add department filter if requested
        if department_filter:
            # Extract unique departments
            departments = set()
            for employee in employees:
                if 'department' in employee:
                    departments.add(employee['department'])
            
            # Create filter dropdown
            if departments:
                choices = []
                for dept in sorted(departments):
                    choices.append({
                        "title": dept,
                        "value": dept
                    })
                
                filter_container = {
                    "type": "Container",
                    "items": [
                        {
                            "type": "TextBlock",
                            "text": "Filter by Department:",
                            "weight": "bolder",
                            "wrap": True
                        },
                        {
                            "type": "Input.ChoiceSet",
                            "id": "department_filter",
                            "choices": choices,
                            "style": "expanded"
                        }
                    ]
                }
                card_data["body"].append(filter_container)
                
                # Add actions array if it doesn't exist
                if "actions" not in card_data:
                    card_data["actions"] = []
                    
                # Add filter button
                card_data["actions"].append({
                    "type": "Action.Submit",
                    "title": "Apply Filter",
                    "data": {"action": "filter_directory"}
                })
        
        # Add employee listing
        employee_listing = {
            "type": "Container",
            "items": [
                {
                    "type": "TextBlock",
                    "text": "Employee Listing",
                    "weight": "bolder",
                    "size": "medium",
                    "wrap": True
                }
            ]
        }
        
        # Add employees (limit to first 5 for demonstration)
        display_employees = employees[:5] if len(employees) > 5 else employees
        
        for i, employee in enumerate(display_employees):
            # Create employee container
            emp_container = {
                "type": "Container",
                "items": []
            }
            
            # Create employee row columns
            emp_columns = []
            
            # Employee photo
            if 'image_url' in employee:
                image_column = {
                    "type": "Column",
                    "width": "auto",
                    "items": [
                        {
                            "type": "Image",
                            "url": employee['image_url'],
                            "size": "small",
                            "style": "person",
                            "altText": f"Photo of {employee.get('name', 'Employee')}"
                        }
                    ]
                }
                emp_columns.append(image_column)
            
            # Employee details
            details_items = []
            
            if 'name' in employee:
                details_items.append({
                    "type": "TextBlock",
                    "text": employee['name'],
                    "weight": "bolder",
                    "wrap": True
                })
            
            if 'title' in employee:
                details_items.append({
                    "type": "TextBlock",
                    "text": employee['title'],
                    "wrap": True
                })
                
            if 'department' in employee:
                details_items.append({
                    "type": "TextBlock",
                    "text": employee['department'],
                    "isSubtle": True,
                    "size": "small",
                    "wrap": True
                })
                
            if 'location' in employee:
                details_items.append({
                    "type": "TextBlock",
                    "text": employee['location'],
                    "isSubtle": True,
                    "size": "small",
                    "wrap": True
                })
            
            details_column = {
                "type": "Column",
                "width": "stretch",
                "items": details_items
            }
            emp_columns.append(details_column)
            
            # Contact info column
            contact_items = []
            
            if 'email' in employee:
                contact_items.append({
                    "type": "TextBlock",
                    "text": employee['email'],
                    "size": "small",
                    "wrap": True
                })
                
            if 'phone' in employee:
                contact_items.append({
                    "type": "TextBlock",
                    "text": employee['phone'],
                    "size": "small",
                    "wrap": True
                })
            
            if contact_items:
                contact_column = {
                    "type": "Column",
                    "width": "auto",
                    "items": contact_items
                }
                emp_columns.append(contact_column)
            
            # Create row and add to container
            emp_row = {
                "type": "ColumnSet",
                "columns": emp_columns
            }
            emp_container["items"].append(emp_row)
            
            # Add view profile action
            action_set = {
                "type": "ActionSet",
                "actions": [
                    {
                        "type": "Action.Submit",
                        "title": "View Profile",
                        "data": {"action": "view_profile", "employee_id": employee.get('id', '')}
                    }
                ]
            }
            emp_container["items"].append(action_set)
            
            # Add separator between employees (except after the last one)
            if i < len(display_employees) - 1:
                emp_container["items"].append({"type": "Separator"})
            
            # Add to employee listing
            employee_listing["items"].append(emp_container)
        
        # Add pagination if needed
        if len(employees) > 5:
            pagination = {
                "type": "Container",
                "items": [
                    {
                        "type": "TextBlock",
                        "text": f"Showing 5 of {len(employees)} employees",
                        "isSubtle": True,
                        "size": "small",
                        "wrap": True
                    }
                ]
            }
            
            # Add pagination actions
            pagination_actions = {
                "type": "ActionSet",
                "actions": [
                    {
                        "type": "Action.Submit",
                        "title": "Previous",
                        "data": {"action": "pagination", "direction": "prev", "page": 1}
                    },
                    {
                        "type": "Action.Submit",
                        "title": "Next",
                        "data": {"action": "pagination", "direction": "next", "page": 1}
                    }
                ]
            }
            
            pagination["items"].append(pagination_actions)
            employee_listing["items"].append(pagination)
        
        # Add the employee listing to the card
        card_data["body"].append(employee_listing)
        
        # Add actions array if it doesn't exist
        if "actions" not in card_data:
            card_data["actions"] = []
            
        # Add view all action
        card_data["actions"].append({
            "type": "Action.Submit",
            "title": "View All Employees",
            "data": {"action": "view_all_employees"}
        })
        
        # Create a card object with to_json method
        class CardObject:
            def __init__(self, card_data):
                self.card_data = card_data
                
            def to_json(self):
                return json.dumps(self.card_data)
        
        return CardObject(card_data)
        
    except Exception as e:
        print(f"Error creating employee directory template: {e}")
        return None

In [ ]:
try:
    # Sample employee data
    employees = [
        {
            "id": "emp001",
            "name": "Jessica Chen",
            "title": "Senior Product Manager",
            "department": "Product",
            "location": "San Francisco, CA",
            "email": "jessica.chen@example.com",
            "phone": "555-123-4567",
            "image_url": "https://adaptivecards.io/content/adaptive-card-50.png"
        },
        {
            "id": "emp002",
            "name": "Marcus Johnson",
            "title": "Software Engineer",
            "department": "Engineering",
            "location": "Seattle, WA",
            "email": "marcus.johnson@example.com",
            "phone": "555-234-5678",
            "image_url": "https://adaptivecards.io/content/adaptive-card-50.png"
        },
        {
            "id": "emp003",
            "name": "Sophia Rodriguez",
            "title": "UX Designer",
            "department": "Design",
            "location": "Austin, TX",
            "email": "sophia.rodriguez@example.com",
            "phone": "555-345-6789",
            "image_url": "https://adaptivecards.io/content/adaptive-card-50.png"
        },
        {
            "id": "emp004",
            "name": "James Wilson",
            "title": "Marketing Director",
            "department": "Marketing",
            "location": "New York, NY",
            "email": "james.wilson@example.com",
            "phone": "555-456-7890",
            "image_url": "https://adaptivecards.io/content/adaptive-card-50.png"
        },
        {
            "id": "emp005",
            "name": "Emily Davis",
            "title": "HR Specialist",
            "department": "Human Resources",
            "location": "Chicago, IL",
            "email": "emily.davis@example.com",
            "phone": "555-567-8901",
            "image_url": "https://adaptivecards.io/content/adaptive-card-50.png"
        },
        {
            "id": "emp006",
            "name": "Michael Brown",
            "title": "Financial Analyst",
            "department": "Finance",
            "location": "Boston, MA",
            "email": "michael.brown@example.com",
            "phone": "555-678-9012",
            "image_url": "https://adaptivecards.io/content/adaptive-card-50.png"
        }
    ]
    
    # Create employee directory card
    directory_card = create_employee_directory_template(
        title="Company Directory",
        employees=employees,
        department_filter=True,
        search_box=True
    )
    
    display_card(directory_card, "Employee Directory Card")
    
except Exception as e:
    print(f"Error creating employee directory card: {e}")

## Note on Common Errors

When implementing Adaptive Cards with this toolkit, be aware of these common errors:

1. **'property' object has no attribute 'schema'** - This appears in AdaptiveCard related operations, specifically when creating cards with AdaptiveCard.create(). The fix is to:
   - Use direct JSON construction as shown in these examples
   - Create a CardObject wrapper with to_json() method
   - Avoid using AdaptiveCard.create() or schema-related methods

2. **ElementFactory.create_text() got an unexpected keyword argument 'wrap'** - The ElementFactory methods don't accept a 'wrap' parameter directly. Instead:
   - Use direct JSON construction as shown in the examples above
   - Set "wrap": True directly in the JSON structure

3. **'NoneType' object has no attribute 'to_json'** - This occurs when card creation fails and returns None, but code tries to use to_json(). To fix:
   - Use a CardObject wrapper with a to_json() method as shown in these examples
   - Ensure exception handling properly checks for None before calling to_json()
   - Follow the Card Creation Pattern from CLAUDE.md that uses direct JSON construction for complex cards

These examples use direct JSON construction to avoid these errors. This approach follows the recommended pattern in the toolkit.

## Key Takeaways

In this notebook, you've learned:

1. How to add rich metadata displays for content
2. How to implement profile cards for detailed personal information
3. How to build team profile cards for displaying multiple team members
4. How to develop employee directory cards for staff lookup
5. How to use direct JSON construction to avoid common errors with the AdaptiveCard library

In the next notebook, we'll explore specialized documentation and information-rich templates.